![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=Mathematics/StatisticsProject/AccessingData/spotify-popularity.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Spotify Music

Using Spotify data will require setting up an [Spotify Web API key](https://developer.spotify.com/documentation/web-api/) that allows us to get information about songs, albums, and artists.

If you have a [Spotify account](https://www.spotify.com/us/signup), you can sign in to the [Developers Dashboard](https://developer.spotify.com/dashboard/login). From the Dashboard, you can click the `CREATE AN APP` button, type a name and description, and then click `CREATE`. Clicking on your new app in the Dashboard will show you the `Client ID` and `CLIENT SECRET` that you can paste into the code cell below.

In [ ]:
CLIENT_ID = 'PASTE_YOUR_CLIENT_ID_HERE'
CLIENT_SECRET = 'PASTE_YOUR_CLIENT_SECRET_HERE'

import requests
import pandas as pd
try:
    auth_response = requests.post('https://accounts.spotify.com/api/token', {'grant_type':'client_credentials', 'client_id':CLIENT_ID, 'client_secret':CLIENT_SECRET})
    auth_response_data = auth_response.json()
    access_token = auth_response_data['access_token']
    headers = {'Authorization':'Bearer {token}'.format(token=access_token)}
except:
    print('Remember to paste your client ID and secret into the code')

def find_tracks(search_string):
    try:
        r = requests.get('https://api.spotify.com/v1/search?q=' + search_string + '&type=track', headers=headers)
        info = r.json()
    except:
        print('Error with search string:', search_string)
        info = None
    return info

def get_track_info(track_id):
    try:
        r = requests.get('https://api.spotify.com/v1/tracks/' + track_id, headers=headers)
        info = r.json()
    except:
        print('Error with track id:', track_id)
        info = None
    return info

def get_track_features(track_id):
    try:
        r = requests.get('https://api.spotify.com/v1/audio-features/' + track_id, headers=headers)
        info = r.json()
    except:
        print('Error with track id:', track_id)
        info = None
    return info

print('Spotify API setup complete')

As an example of a list of songs to get data about, try the [Billions Club Playlist](https://open.spotify.com/playlist/37i9dQZF1DX7iB3RCnBnN4), which has an id of `37i9dQZF1DX7iB3RCnBnN4`, or paste in the id of your own playlist.

In [ ]:
playlist_id = '37i9dQZF1DX7iB3RCnBnN4'

tracks = []
for x in range(4):  # since it only returns 100 tracks at a time and we want to get all of them
    offset = x*100
    r = requests.get('https://api.spotify.com/v1/playlists/' + playlist_id + '/tracks?offset=' + str(offset), headers=headers)
    for item in r.json()['items']:
        tracks.append([item['track']['artists'][0]['name'], item['track']['name'], item['track']['id']])
pl = pd.DataFrame(tracks, columns=['artist', 'track', 'id'])

# get track features
track_features = {}
for row in pl.itertuples():
    print(row[1], row[2]) # artist and track
    id = row[3]
    features = get_track_features(id)
    track_features[id] = features

from IPython.display import clear_output
clear_output()

# merge the data to the track list
tf = pd.DataFrame(track_features).T
playlist = pd.merge(pl, tf, on='id')
playlist

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)